## CS-A1155 Databases for Data Science 

# The Relational Data Model 

Alexander Jung, Associate Professor for Machine Learning, Aalto University  
- **Website**: [click here](https://machinelearningforall.github.io/)
- **LinkedIn**: [linkedin.com/in/aljung](https://www.linkedin.com/in/aljung/)  
- **YouTube**: [youtube.com/@alexjung111](https://www.youtube.com/@alexjung111)

<div style="padding: 15px; border: 1px solid transparent; border-color: transparent; margin-bottom: 20px; border-radius: 4px; color: #3c763d; background-color: #dff0d8; border-color: #d6e9c6;">

## Learning Goals

This module teaches you how
- to use relations to model and reason about datasets. 
- to apply relational algebra to build new relations from given relations.  
- to implement the relational data model using `sqlite3`. 

## Reading
 
- Chapter 2 (Relational Model) of "A First Course in Database Systems" by Jeffrey D. Ullman and Jennifer Widom, available online via Aalto library: https://primo.aalto.fi/permalink/358AALTO_INST/1g8mond/alma999383044506526
- Documentation of `sqlite3` https://docs.python.org/3/library/sqlite3.html
</div>

## Relations

A **relation** (or table) is formally defined as a **set of tuples** (or rows) with attributes (columns). We denote a relation as:

$$ R(A_1: T_1, A_2: T_2, ..., A_n: T_n) $$

where:
- $ R $ is the **name** of the relation (table)
- $ A_1, A_2, ..., A_n $ are **attributes** (columns) of the relation
- $ T_1, T_2, ..., T_n $ are the corresponding **domains** or **data types** of the attributes

For example, consider a relation representing information about Students:

$$ Students(sid: INTEGER, name: STRING, major: STRING, year: INTEGER) $$

where:
- $ sid: INTEGER $ – the **student ID** (Primary Key)
- $ name: STRING $ – stores the student’s **name**
- $ major: STRING $ – stores the student’s **field of study**
- $ year: INTEGER $ – represents the student’s **expected graduation year**
  
### Tuples and Attribute Values

A **tuple** $t$ is a single entry (or row) in a relation $R$. Each tuple consists of **attribute values** corresponding to the attributes of the relation.

If a relation $R$ has attributes $ \{ A_1, A_2, ..., A_n \} $, then a tuple $ t $ in $ R $ is represented as:

$$
t = (v_1, v_2, ..., v_n)
$$

where $ v_i = t[A_i] $ is the value of attribute $ A_i $ in the tuple $ t $. 

To build intuition, it is useful to think of a relation $R$ as a two-dimensional table with cells containing data. The name of the table corresponds to the name of the relation, the columns of the table represent the attributes and the rows of the table represent the tuples. **Note: Relations are a special type of tables in the sense that the order of the rows is irrelevant.**

### Example.
Given the `Students` relation, an example set of tuples could be:

$$
\begin{aligned}
    t_1 &= (1, \text{``Alice''}, \text{``Computer Science''}, 2024) \\
    t_2 &= (2, \text{``Bob''}, \text{``Mathematics''}, 2025)
\end{aligned}
$$

For tuple $ t_1 $:
- $t_1[sid] = 1$
- $t_1[name] = \text{``Alice''}$
- $t_1[major] = \text{``Computer Science''}$
- $t_1[year] = 2024$
- A **subtuple** of $t_1$ on, say, $(sid, major)$ would be $t_1[sid, major] = (1, \text{``Computer Science''})$. 

Each tuple represents a unique entity (a student in this case) and stores specific values for its attributes. Each of these values must belong to the domain of the corresponding attribute, e.g., the value $t_1[major]$ must belong to the domain $STRING$.


---

In [11]:
import sqlite3
from tabulate import tabulate  # Install with: pip install tabulate

# Connect to SQLite database
conn = sqlite3.connect(":memory:")  # Use an in-memory DB for demonstration
cursor = conn.cursor()

# Create the Movies table
cursor.execute('''
    CREATE TABLE Movies (
        title TEXT,
        year INTEGER,
        length INTEGER,
        genre TEXT
    )
''')

# Insert data into the Movies table
movies_data = [
    ("Gone With the Wind", 1939, 231, "drama"),
    ("Star Wars", 1977, 124, "sciFi"),
    ("Wayne’s World", 1992, 95, "comedy")
]

cursor.executemany("INSERT INTO Movies (title, year, length, genre) VALUES (?, ?, ?, ?)", movies_data)

# Retrieve and display the table contents
cursor.execute("SELECT * FROM Movies")
movies = cursor.fetchall()

# Print table using tabulate
headers = ["Title", "Year", "Length (min)", "Genre"]
print("\nMovies Table:")
print(tabulate(movies, headers=headers, tablefmt="grid"))

# Commit and close the connection
conn.commit()
conn.close()

print("\nMovies database created, populated, and displayed successfully.")


Movies Table:
+--------------------+--------+----------------+---------+
| Title              |   Year |   Length (min) | Genre   |
+====================+========+================+=========+
| Gone With the Wind |   1939 |            231 | drama   |
+--------------------+--------+----------------+---------+
| Star Wars          |   1977 |            124 | sciFi   |
+--------------------+--------+----------------+---------+
| Wayne’s World      |   1992 |             95 | comedy  |
+--------------------+--------+----------------+---------+

Movies database created, populated, and displayed successfully.


### Relation Instances 

While the schema of a relation specifies the overall structure—including its name, attributes, and data types—the actual content of a relation changes frequently as data is inserted, updated, or removed. A **relation instance** represents the specific set of tuples currently stored in a relation at a particular moment in time. You could think of an instance as a snapshot of the dataset represented by a relation. 

Formally, given a relation $ R(A_1: T_1, A_2: T_2, \dots, A_n: T_n) $, an instance of this relation is a set of tuples:

$$
\{ t_1, t_2, \dots, t_m \}
$$

where each tuple $ t_i $ provides a concrete value $t_i[A_j]$ from the respective domain $ T_j $ for each attribute $ A_j $.

### Example. 
Consider the `Students` relation:

$
Students(sid: INTEGER, name: STRING, major: STRING, year: INTEGER)
$

An instance of the `Students` relation could be:

$
\begin{aligned}
    t_1 &= (1, \text{``Alice''}, \text{``Computer Science''}, 2024) \\
    t_2 &= (2, \text{``Bob''}, \text{``Mathematics''}, 2025)
\end{aligned}
$

This set of tuples represents the **current instance** of the relation. The contents of this instance will change over time as new students enroll, existing student details are updated, or students graduate and their records are removed.

You can **add tuples** to obtain new relation instances. For example, adding a new tuple:

$$
t_3 = (3, \text{``Charlie''}, \text{``Physics''}, 2026)
$$

results in an updated instance:

$$
\{ t_1, t_2, t_3 \}
$$

Similarly, you can **remove tuples**. For example, removing tuple $ t_2 $ from the relation results in another instance:

$$
\{ t_1, t_3 \}
$$

Note that while relation instances change frequently, the schema itself—defining the structure and data types—is generally stable and rarely modified. Schema modifications, though possible, are usually costly as they require extensive updates to existing data.

In [12]:
import sqlite3
from tabulate import tabulate  # pip install tabulate

# Connect to SQLite database in memory for demonstration purposes
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create the Students relation schema
cursor.execute('''
    CREATE TABLE Students (
        sid INTEGER PRIMARY KEY,
        name TEXT,
        major TEXT,
        year INTEGER
    )
''')

# Insert initial tuples into Students (creating the initial instance)
students_data = [
    (1, "Alice", "Computer Science", 2024),
    (2, "Bob", "Mathematics", 2025)
]
cursor.executemany("INSERT INTO Students (sid, name, major, year) VALUES (?, ?, ?, ?)", students_data)

# Function to display current instance of Students relation
def display_students(title):
    cursor.execute("SELECT * FROM Students")
    students = cursor.fetchall()
    print(f"\n{title}:")
    print(tabulate(students, headers=["sid", "name", "major", "year"], tablefmt="grid"))

# Display initial instance
display_students("Initial Students Relation Instance")

# Add a new tuple (student) to create a new instance
new_student = (3, "Charlie", "Physics", 2026)
cursor.execute("INSERT INTO Students (sid, name, major, year) VALUES (?, ?, ?, ?)", new_student)

# Display updated instance after addition
display_students("Instance after Adding a Tuple (Charlie)")

# Remove a tuple (student with sid=2, Bob) to create another instance
cursor.execute("DELETE FROM Students WHERE sid = ?", (2,))

# Display updated instance after deletion
display_students("Instance after Removing a Tuple (Bob)")

# Close the connection
conn.close()



Initial Students Relation Instance:
+-------+--------+------------------+--------+
|   sid | name   | major            |   year |
+=======+========+==================+========+
|     1 | Alice  | Computer Science |   2024 |
+-------+--------+------------------+--------+
|     2 | Bob    | Mathematics      |   2025 |
+-------+--------+------------------+--------+

Instance after Adding a Tuple (Charlie):
+-------+---------+------------------+--------+
|   sid | name    | major            |   year |
+=======+=========+==================+========+
|     1 | Alice   | Computer Science |   2024 |
+-------+---------+------------------+--------+
|     2 | Bob     | Mathematics      |   2025 |
+-------+---------+------------------+--------+
|     3 | Charlie | Physics          |   2026 |
+-------+---------+------------------+--------+

Instance after Removing a Tuple (Bob):
+-------+---------+------------------+--------+
|   sid | name    | major            |   year |
+=======+=========+=====

---
### Keys

Consider some **relation** $R$ which consists of several tuples $t$. We refer to a **set of attributes** $\{A_1, A_2, \dots, A_k\}$ of $R$ as a **key** if it uniquely identifies each **tuple** $t$ in a **relation** $R$. More precisely: The attributes $\{A_1, A_2, \dots, A_k\}$ form a primary key of a relation $R$ if 

$$ \forall t_1, t_2 \in R, \left( \forall A_i \in \{A_1, A_2, \dots, A_k\}, t_1[A_i] = t_2[A_i] \right) \Rightarrow (t_1 = t_2) $$

In other words, if two tuples $t_1$ and $t_2$ have the same values for all attributes in the primary key, then they must be the same tuple. Two examples: 
- in the **Students** relation $\text{Students}$ the attribute $\text{sid}$ is a key. Indeed, each tuple $t$ in **Students** has a unique value for **sid**, ensuring that no two students share the same identifier.
- in a relation $\text{CourseEnrollments}$ storing information about course enrollments, a key could be $\big\{ \text{sid},\text{courseid} \big\}$. Indeed, a student can enroll in multiple courses but cannot enroll in the same course more than once.

Strictly speaking, a key must also satisfy a minimality property: It must not contain a proper subset of attributes that can serve as key. In general, there can be several choices for the key of a relation. It can then be useful to choose one of them as the **primary key**.

In [13]:
import sqlite3

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create Students table with sid as the primary key
# A primary key ensures that each entry in the table is uniquely identifiable.
# The column 'sid' is chosen as the primary key, meaning no two 
# students can have the same sid.

cursor.execute('''
    CREATE TABLE Students (
        sid INTEGER PRIMARY KEY,  
        name TEXT NOT NULL,
        age INTEGER
    )
''')

# Insert some students
cursor.execute("INSERT INTO Students (sid, name, age) VALUES (1, 'Alice', 22)")
cursor.execute("INSERT INTO Students (sid, name, age) VALUES (2, 'Bob', 24)")

# Attempting to insert a student with duplicate sid should fail
try:
    cursor.execute("INSERT INTO Students (sid, name, age) VALUES (1, 'Charlie', 23)")
except sqlite3.IntegrityError:
    print("IntegrityError: Cannot insert duplicate primary key (sid=1)")

# Create CourseEnrollments table with a composite primary key (sid, courseid)
# A composite primary key consists of multiple columns that together uniquely identify each row.
# Here, (sid, courseid) ensures that a student can enroll in multiple courses,
# but cannot enroll in the same course more than once.
cursor.execute('''
    CREATE TABLE CourseEnrollments (
        sid INTEGER,
        courseid TEXT,
        grade TEXT,
        PRIMARY KEY (sid, courseid)
    )
''')

# Insert valid course enrollments
cursor.execute("INSERT INTO CourseEnrollments (sid, courseid, grade) VALUES (1, 'CS101', 'A')")
cursor.execute("INSERT INTO CourseEnrollments (sid, courseid, grade) VALUES (1, 'CS102', 'B')")
cursor.execute("INSERT INTO CourseEnrollments (sid, courseid, grade) VALUES (2, 'CS101', 'A')")

# Attempting to insert a duplicate (sid, courseid) pair should fail
try:
    cursor.execute("INSERT INTO CourseEnrollments (sid, courseid, grade) VALUES (1, 'CS101', 'B')")
except sqlite3.IntegrityError:
    print("IntegrityError: Cannot insert duplicate primary key (sid=1, courseid='CS101')")

# Fetch and display results
cursor.execute("SELECT * FROM Students")
print("\nStudents Table:")
for row in cursor.fetchall():
    print(row)

cursor.execute("SELECT * FROM CourseEnrollments")
print("\nCourseEnrollments Table:")
for row in cursor.fetchall():
    print(row)

# Close connection
conn.close()


IntegrityError: Cannot insert duplicate primary key (sid=1)
IntegrityError: Cannot insert duplicate primary key (sid=1, courseid='CS101')

Students Table:
(1, 'Alice', 22)
(2, 'Bob', 24)

CourseEnrollments Table:
(1, 'CS101', 'A')
(1, 'CS102', 'B')
(2, 'CS101', 'A')


### Foreign Key (FK)

A **foreign key (FK)** is an attribute (or set of attributes) of a relation $R$ that establishes a link to the **primary key (PK)** in another relation $S$. Every FK value in $R$ must refer to an existing PK value in $S$. This ensures consistency in the database. Formally, given a relation $R$ that contains a foreign key $FK$ referencing a primary key $PK$ in another relation $S$:

$$
\forall t \in R, \quad t[FK] \in \{ s[PK] \mid s \in S \}
$$

This means that for every tuple $t$ in relation $R$, the value of the foreign key $t[FK]$ must exist as a primary key value in relation $S$, i.e., there must be some $s \in S$ such that $s[PK]=t[FK]$.  

---

In [14]:
import sqlite3

# Connect to SQLite database (or create it)
conn = sqlite3.connect(":memory:")  # Use in-memory DB for demo purposes
cursor = conn.cursor()

# Enable foreign key constraints in SQLite
cursor.execute("PRAGMA foreign_keys = ON;")

# Create the Students table with sid as the Primary Key
cursor.execute('''
    CREATE TABLE Students (
        sid INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        major TEXT NOT NULL
    )
''')

# Create the CourseEnrollments table with sid as a Foreign Key referencing Students.sid
cursor.execute('''
    CREATE TABLE CourseEnrollments (
        enrollment_id INTEGER PRIMARY KEY AUTOINCREMENT,
        sid INTEGER,
        course_id TEXT NOT NULL,
        FOREIGN KEY (sid) REFERENCES Students(sid) ON DELETE CASCADE
    )
''')

# Insert some student records
cursor.executemany("INSERT INTO Students (sid, name, major) VALUES (?, ?, ?)", [
    (1, "Alice", "Computer Science"),
    (2, "Bob", "Mathematics")
])

# Try inserting a valid enrollment (Alice exists in Students)
cursor.execute("INSERT INTO CourseEnrollments (sid, course_id) VALUES (?, ?)", (1, "CS101"))

# Try inserting an invalid enrollment (sid=3 does not exist in Students, should fail)
try:
    cursor.execute("INSERT INTO CourseEnrollments (sid, course_id) VALUES (?, ?)", (3, "CS102"))
except sqlite3.IntegrityError as e:
    print("Error:", e)

# Retrieve and display all enrollments
cursor.execute("SELECT * FROM CourseEnrollments")
enrollments = cursor.fetchall()
print("\nValid enrollments in CourseEnrollments:")
for row in enrollments:
    print(row)

# Commit and close connection
conn.commit()
conn.close()


Error: FOREIGN KEY constraint failed

Valid enrollments in CourseEnrollments:
(1, 1, 'CS101')


### Relation Schema

The **schema** of a relation defines its structure, specifying the attributes and their data types. For example, the schema of the **Students** relation is:

$$ Students(sid: INTEGER, name: TEXT, major: TEXT, year: INTEGER) $$

Another example is the schema of the **Movies** relation:

$$ Movies(title: TEXT, year: INTEGER, length: INTEGER, genre: TEXT) $$

The schema of a relation starts with the name of the relation followed by parentheses and a list of its attributes in a certain order. Although the attributes of a relation are a set, the order specified is considered the “standard” order whenever the schema of that relation is mentioned.

### Database Schema

A **database schema** consists of multiple relation schemas that define the structure of a database. It specifies how different relations interact with each other using **primary keys** and **foreign keys**.

For example, consider a database schema for a university system that includes **Students**, **Courses**, and **Enrollments** relations:

$$ Students(sid: INTEGER, name: TEXT, major: TEXT, year: INTEGER) $$  
$$ Courses(course_id: TEXT, title: TEXT, credits: INTEGER) $$  
$$ Enrollments(enrollment_id: INTEGER, sid: INTEGER, course_id: TEXT) $$  

- `sid` is the **primary key** of the **Students** relation.
- `course_id` is the **primary key** of the **Courses** relation.
- In the **Enrollments** relation:
  - `enrollment_id` is the **primary key**.
  - `sid` is a **foreign key** referencing `Students.sid`.
  - `course_id` is a **foreign key** referencing `Courses.course_id`.

This schema ensures that every enrollment record references a valid student and a valid course, maintaining referential integrity in the database.


## Relational Algebra

**Relational algebra** is a formal language that allows to construct new relations from existing ones through a set of fundamental operations. We can think of these new relations as "answers" to database queries which are formulated as algebraic expressions. Just like arithmetic algebra uses operands (variables and constants) and operators (e.g., addition, multiplication) to produce new numerical expressions, relational algebra constructs new relations from existing ones using relational operations.

Relational algebra serves as the theoretical foundation for SQL and other query languages, providing a formal method for precisely defining database queries and transformations. Understanding relational algebra is beneficial not only because of its theoretical clarity, but also because it provides valuable intuition for mastering SQL and other database query languages built upon the relational model.

Consider relational algebra and SQL as related yet distinct languages—similar to how German and Dutch are closely related. Both German and Dutch share common roots, structure, and vocabulary, allowing speakers of one to understand much of the other. Likewise, relational algebra and SQL express similar operations and logic for querying databases, yet differ in syntax and style. Relational algebra is more formal, concise, while SQL offers a somewhat more verbose and ``user-friendly'' query writing and execution.

Below is a concise, point-by-point overview of the main operations introduced in **Section 2.4** of *A First Course in Database Systems* (3rd ed.) by Ullman and Widom. For each operation we provide

1. A brief explanation, followed by a 
2. **Python** demo that implements the operation using `sqlite3`.


### Set Operations (Section 2.4.4, *A First Course in Database Systems*, Ullman and Widom, 3rd ed.)

Consider two relations $R, S$ that have the same set of attributes and corresponding domains. Since relations are sets of tuples, we can form their

- **Union** ($R \cup S$) **combines** the tuples from both operand relations, **removing duplicates**.
- **Intersection** ($ R \cap S$) **returns** only those tuples **common** to both operand relations.
- **Difference** ($ R -S$ ) **returns** the tuples **in the first operand** $R$ that are **not** in the second operand $S$.

We emphasize that, for these operations to work properly, 

1. The **relations** must have an **identical set of attributes**.  
2. **Attributes** must belong to the **same domain**.  

In [15]:
import sqlite3
from tabulate import tabulate

# Connect to an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

###################################
# 1) CREATE TABLES
###################################
cursor.execute("""
    CREATE TABLE MoviesR(
        Title TEXT,
        Year INT,
        Genre TEXT
    )
""")
cursor.execute("""
    CREATE TABLE MoviesS(
        Title TEXT,
        Year INT,
        Genre TEXT
    )
""")

###################################
# 2) INSERT SAMPLE DATA
###################################
r_data = [
    ("Inception", 2010, "Sci-Fi"),
    ("The Matrix", 1999, "Sci-Fi"),
    ("Casablanca", 1942, "Drama")
]
s_data = [
    ("The Matrix", 1999, "Sci-Fi"),  # Overlap with MoviesR
    ("Titanic", 1997, "Romance"),
    ("Alien", 1979, "Sci-Fi")
]

cursor.executemany("INSERT INTO MoviesR VALUES (?,?,?)", r_data)
cursor.executemany("INSERT INTO MoviesS VALUES (?,?,?)", s_data)

###################################
# 3) DISPLAY BOTH TABLES
###################################
def fetch_and_print_table(table_name):
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    print(f"\n{table_name} Table:")
    print(tabulate(rows, headers=["Title","Year","Genre"], tablefmt="grid"))

fetch_and_print_table("MoviesR")
fetch_and_print_table("MoviesS")

###################################
# 4) UNION
###################################
cursor.execute("""
    SELECT * FROM MoviesR
    UNION
    SELECT * FROM MoviesS
""")
union_result = cursor.fetchall()

print("\n--- UNION (MoviesR ∪ MoviesS) ---")
print(tabulate(union_result, headers=["Title","Year","Genre"], tablefmt="grid"))

###################################
# 5) INTERSECTION
###################################
cursor.execute("""
    SELECT * FROM MoviesR
    INTERSECT
    SELECT * FROM MoviesS
""")
intersection_result = cursor.fetchall()

print("\n--- INTERSECTION (MoviesR ∩ MoviesS) ---")
print(tabulate(intersection_result, headers=["Title","Year","Genre"], tablefmt="grid"))

###################################
# 6) DIFFERENCE (MoviesR - MoviesS)
###################################
cursor.execute("""
    SELECT * FROM MoviesR
    EXCEPT
    SELECT * FROM MoviesS
""")
difference_result = cursor.fetchall()

print("\n--- DIFFERENCE (MoviesR - MoviesS) ---")
print(tabulate(difference_result, headers=["Title","Year","Genre"], tablefmt="grid"))

# Close the connection
conn.close()


MoviesR Table:
+------------+--------+---------+
| Title      |   Year | Genre   |
+============+========+=========+
| Inception  |   2010 | Sci-Fi  |
+------------+--------+---------+
| The Matrix |   1999 | Sci-Fi  |
+------------+--------+---------+
| Casablanca |   1942 | Drama   |
+------------+--------+---------+

MoviesS Table:
+------------+--------+---------+
| Title      |   Year | Genre   |
+============+========+=========+
| The Matrix |   1999 | Sci-Fi  |
+------------+--------+---------+
| Titanic    |   1997 | Romance |
+------------+--------+---------+
| Alien      |   1979 | Sci-Fi  |
+------------+--------+---------+

--- UNION (MoviesR ∪ MoviesS) ---
+------------+--------+---------+
| Title      |   Year | Genre   |
+============+========+=========+
| Alien      |   1979 | Sci-Fi  |
+------------+--------+---------+
| Casablanca |   1942 | Drama   |
+------------+--------+---------+
| Inception  |   2010 | Sci-Fi  |
+------------+--------+---------+
| The Matrix | 

### Projection Operator (Section 2.4.5, *A First Course in Database Systems*, Ullman & Widom, 3rd ed.)

The **projection** operator $\pi$ **selects** a subset of attributes from a relation, discarding the rest. For a relation $R$ with $n$ attributes, the projection on the subset $A_1,\ldots,A_{k}$ of $k$ attributes is defined as 

$$
\pi_{A_1, A_2, \ldots, A_k}(R)
=
\{\,
t[A_1, A_2, \ldots, A_k]
\;\mid\;
t \in R
\}
$$

where:
- $ R $ is a relation with $n$ attributes, 
- $A_1, A_2, \ldots, A_k$ are **attributes** (column names) of $R$.
- $t[A_1, A_2, \ldots, A_k]$ denotes the **subtuple** of $t$ containing only attributes $A_1, \ldots, A_k$.

In words, “take each tuple $t$ in $R$, but **keep** only the values from the specified attributes.”

In [16]:
import sqlite3
from tabulate import tabulate

# --------------------------------------------------------
# 1) Connect to an in-memory SQLite database
# --------------------------------------------------------
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# --------------------------------------------------------
# 2) Create a 'Students' table
# --------------------------------------------------------
cursor.execute("""
    CREATE TABLE Students(
        sid   INTEGER,
        name  TEXT,
        major TEXT,
        year  INT
    )
""")

# --------------------------------------------------------
# 3) Insert sample rows, including repeated (major, year) 
#    so we can demonstrate duplicates
# --------------------------------------------------------
students_data = [
    (1, "Alice",   "CS",   2024),
    (2, "Bob",     "CS",   2024),  # Same (major, year) as Alice
    (3, "Carol",   "Math", 2024),
    (4, "Diana",   "CS",   2024),  # Same (major, year) as Alice
]
cursor.executemany("INSERT INTO Students VALUES (?,?,?,?)", students_data)

# --------------------------------------------------------
# 4) Show the original table
# --------------------------------------------------------
cursor.execute("SELECT * FROM Students")
original_rows = cursor.fetchall()
print("=== Original Table (Students) ===")
print(tabulate(original_rows, headers=["sid","name","major","year"], tablefmt="grid"))

# --------------------------------------------------------
# 5) Projection in standard SQL vs. 'Relational Algebra' (distinct)
# --------------------------------------------------------

# (A) Simple SELECT: does NOT remove duplicates by default
cursor.execute("SELECT major, year FROM Students")
proj_with_duplicates = cursor.fetchall()

# simulates set-theoretic projection (removes duplicates)
cursor.execute("SELECT DISTINCT major, year FROM Students")
proj_distinct = cursor.fetchall()

print("\n=== SQL SELECT (major, year) WITHOUT removing duplicates ===")
print(tabulate(proj_with_duplicates, headers=["major","year"], tablefmt="grid"))

print("\n=== Projection (major, year) WITH duplicates removed (DISTINCT) ===")
print(tabulate(proj_distinct, headers=["major","year"], tablefmt="grid"))

# --------------------------------------------------------
# 6) Close the connection
# --------------------------------------------------------
conn.close()


=== Original Table (Students) ===
+-------+--------+---------+--------+
|   sid | name   | major   |   year |
+=======+========+=========+========+
|     1 | Alice  | CS      |   2024 |
+-------+--------+---------+--------+
|     2 | Bob    | CS      |   2024 |
+-------+--------+---------+--------+
|     3 | Carol  | Math    |   2024 |
+-------+--------+---------+--------+
|     4 | Diana  | CS      |   2024 |
+-------+--------+---------+--------+

=== SQL SELECT (major, year) WITHOUT removing duplicates ===
+---------+--------+
| major   |   year |
+=========+========+
| CS      |   2024 |
+---------+--------+
| CS      |   2024 |
+---------+--------+
| Math    |   2024 |
+---------+--------+
| CS      |   2024 |
+---------+--------+

=== Projection (major, year) WITH duplicates removed (DISTINCT) ===
+---------+--------+
| major   |   year |
+=========+========+
| CS      |   2024 |
+---------+--------+
| Math    |   2024 |
+---------+--------+


### Selection Operator (Section 2.4.6, *A First Course in Database Systems*, Ullman & Widom, 3rd ed.)

The **selection** operator ($\sigma$) picks out **all tuples** from a relation $R$ that satisfy a specified **condition** (Boolean predicate). Formally, for a condition $\theta$:

$$
\sigma_{\theta}(R)
=
\{\,
t
\;\mid\;
t \in R \text{ and } t \text{ satisfies } \theta
\}
$$

where:
- $R$ is a relation (set of tuples),
- $\theta$ is a condition (e.g., an equality/inequality on attribute values),
- and $t$ denotes an individual tuple of $R$.

In everyday terms, “**filter** the rows of \(R\) to keep only those that make the condition \(\theta\) true.”

---

### Example. 

If $R$ has attributes $(sid, name, major, year)$ and $\theta$ is $\text{major} = \text{"CS"} \land \text{year} \geq 2024$, then:

$$
\sigma_{\text{major} = \text{``CS''} \,\wedge\, \text{year} \geq 2024}(R)
$$

would pick only those **students** who have `major = "CS"` *and* `year >= 2024`.

---

In [17]:
import sqlite3
from tabulate import tabulate

# 1) Connect to an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 2) Create a Students table
cursor.execute("""
    CREATE TABLE Students (
        sid   INTEGER,
        name  TEXT,
        major TEXT,
        year  INT
    )
""")

# 3) Insert sample rows
students_data = [
    (1, "Alice",   "CS",   2023),
    (2, "Bob",     "Math", 2025),
    (3, "Carol",   "CS",   2023),
    (4, "Diana",   "CS",   2024)
]
cursor.executemany("INSERT INTO Students VALUES (?,?,?,?)", students_data)

# 4) Show the original table
cursor.execute("SELECT * FROM Students")
original_rows = cursor.fetchall()
print("=== Original Students Table ===")
print(tabulate(original_rows, headers=["sid","name","major","year"], tablefmt="grid"))

# 5) Selection: major = 'CS' AND year >= 2024
cursor.execute("""
    SELECT *
    FROM Students
    WHERE major='CS' AND year >= 2024
""")
selection_rows = cursor.fetchall()
print("\n=== Selection: major = 'CS' AND year >= 2024 ===")
print(tabulate(selection_rows, headers=["sid","name","major","year"], tablefmt="grid"))

# 6) Close the connection
conn.close()


=== Original Students Table ===
+-------+--------+---------+--------+
|   sid | name   | major   |   year |
+=======+========+=========+========+
|     1 | Alice  | CS      |   2023 |
+-------+--------+---------+--------+
|     2 | Bob    | Math    |   2025 |
+-------+--------+---------+--------+
|     3 | Carol  | CS      |   2023 |
+-------+--------+---------+--------+
|     4 | Diana  | CS      |   2024 |
+-------+--------+---------+--------+

=== Selection: major = 'CS' AND year >= 2024 ===
+-------+--------+---------+--------+
|   sid | name   | major   |   year |
+=======+========+=========+========+
|     4 | Diana  | CS      |   2024 |
+-------+--------+---------+--------+


### Cartesian Product (Section 2.4.7, *A First Course in Database Systems*, Ullman & Widom, 3rd ed.)

Consider two relations $R$ and $S$ with different sets of attributes. The **Cartesian product** operator ($\times$) in relational algebra combines **every tuple** of one relation $R$ with **every tuple** of another relation $S$. If $R$ has attributes $(A_1, A_2, \dots, A_m)$ and $S$ has attributes $(B_1, B_2, \dots, B_n)$, then:

$$
R \times S =
\{
\, (a_1, a_2, \dots, a_m, b_1, b_2, \dots, b_n) 
\;\mid\;
(a_1, \dots, a_m) \in R \;\text{and}\; (b_1, \dots, b_n) \in S
\}.
$$

- **Informally**: “Take **all** possible pairs of rows—one from $R$ and one from $S$—and place them side-by-side.”
- **Result schema**: If
  $$
    \text{Schema}(R) = (A_1, \ldots, A_m), 
    \quad
    \text{Schema}(S) = (B_1, \ldots, B_n),
  $$
  then 
  $$
    \text{Schema}(R \times S) = (A_1, \ldots, A_m,\; B_1, \ldots, B_n).
  $$

### Handling Common Attributes

When $R$ and $S$ have **common attribute names**, the **Cartesian product** can create **duplicate column names** in the result:

- In **relational algebra**, we typically handle this with **rename** ($\rho$).  
- In **SQL**, you can avoid collisions by **aliasing** columns in your query, for example:

###  Example.

If
$$
R = \{\,(\text{Alice},\, \text{CS}),\, (\text{Bob},\, \text{Math})\}
$$
and
$$
S = \{\,(101,\,\text{"Data Structures"}),\, (102,\,\text{"Algebra"})\}
$$
then
$$
R \times S \;=\;
\Bigl\{
   (\text{Alice},\, \text{CS},\, 101,\, \text{"Data Structures"}), \\
   (\text{Alice},\, \text{CS},\, 102,\, \text{"Algebra"}), \\
   (\text{Bob},\, \text{Math},\, 101,\, \text{"Data Structures"}), \\
   (\text{Bob},\, \text{Math},\, 102,\, \text{"Algebra"})
\Bigr\}.
$$


In [18]:
import sqlite3
from tabulate import tabulate

# 1) Connect to an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 2) Create two tables, R and S, each having "name" as a common attribute
cursor.execute("""
    CREATE TABLE R(
        rid   INTEGER,
        name  TEXT,     -- Common attribute
        major TEXT
    )
""")
cursor.execute("""
    CREATE TABLE S(
        cid   INTEGER,
        name  TEXT      -- Common attribute
    )
""")

# 3) Insert sample rows in R
r_data = [
    (1, "Alice", "CS"),
    (2, "Bob",   "Math")
]
cursor.executemany("INSERT INTO R VALUES (?,?,?)", r_data)

# 4) Insert sample rows in S
s_data = [
    (101, "Data Structures"),
    (102, "Algebra")
]
cursor.executemany("INSERT INTO S VALUES (?,?)", s_data)

# 5) Display the two original tables
cursor.execute("SELECT * FROM R")
rows_r = cursor.fetchall()
print("=== Table R ===")
print(tabulate(rows_r, headers=["rid","name","major"], tablefmt="grid"))

cursor.execute("SELECT * FROM S")
rows_s = cursor.fetchall()
print("\n=== Table S ===")
print(tabulate(rows_s, headers=["cid","name"], tablefmt="grid"))

# 6) Cartesian Product (R x S) with attribute aliasing to avoid collision of 'name'
cursor.execute("""
    SELECT 
        R.rid,
        R.name    AS r_name,
        R.major,
        S.cid,
        S.name    AS s_name
    FROM R, S
    -- No WHERE clause => full Cartesian product
""")
cart_product = cursor.fetchall()

print("\n=== Cartesian Product (R x S), with column aliasing ===")
print(tabulate(
    cart_product,
    headers=["rid","r_name","major","cid","s_name"],
    tablefmt="grid"
))

# 7) Close the connection
conn.close()


=== Table R ===
+-------+--------+---------+
|   rid | name   | major   |
+=======+========+=========+
|     1 | Alice  | CS      |
+-------+--------+---------+
|     2 | Bob    | Math    |
+-------+--------+---------+

=== Table S ===
+-------+-----------------+
|   cid | name            |
+=======+=================+
|   101 | Data Structures |
+-------+-----------------+
|   102 | Algebra         |
+-------+-----------------+

=== Cartesian Product (R x S), with column aliasing ===
+-------+----------+---------+-------+-----------------+
|   rid | r_name   | major   |   cid | s_name          |
+=======+==========+=========+=======+=================+
|     1 | Alice    | CS      |   101 | Data Structures |
+-------+----------+---------+-------+-----------------+
|     1 | Alice    | CS      |   102 | Algebra         |
+-------+----------+---------+-------+-----------------+
|     2 | Bob      | Math    |   101 | Data Structures |
+-------+----------+---------+-------+-----------------

### Natural Join (Section 2.4.8, *A First Course in Database Systems*, Ullman & Widom, 3rd ed.)

Consider two relations $R$ and $S$. The **natural join**, denoted as $R \bowtie S$, is a relation with attributes from $R$ and $S$. Each tuple of this new relation is a combination of a tuple $t$ from $R$ and another tuple $u$ from $S$. The tuples $t$ and $u$ must agree on all attributes that $R$ and $S$ have in common. The common attributes appear **only once** in $R \bowtie S$.

In particular, the computation of the natural join involves the following steps:

1. **Identify** the attributes that appear in **both** $R$ and $S$ with the **same** name.
2. **Match** tuples from $R$ and $S$ whose values **agree** on *all* these shared attributes.
3. **Combine** matching tuples into one row, including:
   - All attributes from $R$,
   - All attributes from $S$, except any duplicates for the shared attributes.

---

## Example

Suppose you have two relations:

- $Students$ with attributes $\{sid, name, major\}$.  
- $Enrollments$ with attributes $\{sid, course, grade\}$.

They share the attribute $sid$. A **natural join** $Students \bowtie Enrollments$ will:

1. **Match** rows with the **same** $sid$.  
2. Produce tuples with attributes $\{sid, name, major, course, grade\}$, but $sid$ appears **only once**.

For instance, if:

$$
Students = 
\{(1, \text{"Alice"}, \text{"CS"}), (2, \text{"Bob"}, \text{"Math"})\},
$$

$$
Enrollments = 
\{(1, \text{"CS101"}, \text{"A"}), (2, \text{"CS101"}, \text{"B"}), (2, \text{"Math202"}, \text{"A"})\},
$$

then

$$
Students \bowtie Enrollments = 
\{(1, \text{"Alice"}, \text{"CS"}, \text{"CS101"}, \text{"A"}),
  (2, \text{"Bob"}, \text{"Math"}, \text{"CS101"}, \text{"B"}),
  (2, \text{"Bob"}, \text{"Math"}, \text{"Math202"}, \text{"A"})\}.
$$

---

### Key Points

1. **Common Attributes**  
   - The natural join uses **all** identically named attributes for matching.  
   - If there are **no** such attributes, the natural join essentially becomes a **Cartesian product** of $R$ and $S$.

2. **No Duplicate Columns**  
   - Shared attributes appear **once** in the result, avoiding redundancy.

3. **Naming Pitfalls**  
   - Natural join is handy when the same attribute name truly represents the **same concept** (e.g. `sid` = student ID).  
   - If the same name is used for **different** concepts, then a natural join can yield incorrect matches; use rename operators to fix that problem.

---

### Relationship to Other Operations

A natural join can be seen as:

1. A **Cartesian product** $R \times S$.  
2. A **selection** that enforces **equality** on all attributes sharing the same names.  
3. A **projection** that removes the duplicate columns corresponding to those matches.

Symbolically:

$$
R \bowtie S 
\;=\;
\pi_{\text{all attrs}} 
\bigl(
  \sigma_{\text{common attrs}} \bigl(R \times S\bigr)
\bigr).
$$

If you need more explicit control, you can use a join with a specific condition or rename attributes. But when naming conventions are consistent, **natural join** is concise and expressive.


In [19]:
import sqlite3
from tabulate import tabulate

# 1) Connect to an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 2) Create two tables with a common attribute 'sid'
#    so that a NATURAL JOIN can match on it
cursor.execute("""
    CREATE TABLE Students(
        sid   INTEGER,
        name  TEXT,
        major TEXT
    )
""")
cursor.execute("""
    CREATE TABLE Enrollments(
        sid    INTEGER,
        course TEXT,
        grade  TEXT
    )
""")

# 3) Insert sample rows into Students
students_data = [
    (1, "Alice", "CS"),
    (2, "Bob",   "Math")
]
cursor.executemany("INSERT INTO Students VALUES (?,?,?)", students_data)

# 4) Insert sample rows into Enrollments
enrollments_data = [
    (1, "CS101",   "A"),
    (2, "CS101",   "B"),
    (2, "Math202", "A")
]
cursor.executemany("INSERT INTO Enrollments VALUES (?,?,?)", enrollments_data)

# 5) Display the original tables
cursor.execute("SELECT * FROM Students")
students_rows = cursor.fetchall()
print("=== Students Table ===")
print(tabulate(students_rows, headers=["sid","name","major"], tablefmt="grid"))

cursor.execute("SELECT * FROM Enrollments")
enrollments_rows = cursor.fetchall()
print("\n=== Enrollments Table ===")
print(tabulate(enrollments_rows, headers=["sid","course","grade"], tablefmt="grid"))

# 6) Perform a NATURAL JOIN on 'sid'
#    SQLite recognizes the common column name 'sid'
cursor.execute("""
    SELECT *
    FROM Students NATURAL JOIN Enrollments
""")
natural_join_rows = cursor.fetchall()

print("\n=== NATURAL JOIN (Students NATURAL JOIN Enrollments) ===")
# The result should merge the shared sid column once,
# with columns from both tables in the result.
# We'll figure out the columns from the join's first row if present:
# Or simply supply custom headers if known (sid, name, major, course, grade).
headers = ["sid","name","major","course","grade"]
print(tabulate(natural_join_rows, headers=headers, tablefmt="grid"))

# 7) Close the connection
conn.close()


=== Students Table ===
+-------+--------+---------+
|   sid | name   | major   |
+=======+========+=========+
|     1 | Alice  | CS      |
+-------+--------+---------+
|     2 | Bob    | Math    |
+-------+--------+---------+

=== Enrollments Table ===
+-------+----------+---------+
|   sid | course   | grade   |
+=======+==========+=========+
|     1 | CS101    | A       |
+-------+----------+---------+
|     2 | CS101    | B       |
+-------+----------+---------+
|     2 | Math202  | A       |
+-------+----------+---------+

=== NATURAL JOIN (Students NATURAL JOIN Enrollments) ===
+-------+--------+---------+----------+---------+
|   sid | name   | major   | course   | grade   |
+=======+========+=========+==========+=========+
|     1 | Alice  | CS      | CS101    | A       |
+-------+--------+---------+----------+---------+
|     2 | Bob    | Math    | CS101    | B       |
+-------+--------+---------+----------+---------+
|     2 | Bob    | Math    | Math202  | A       |
+-------+-

### Rename Operator (Section 2.4.11, *A First Course in Database Systems*, Ullman & Widom, 3rd ed.)

The **rename** operator (denoted by $\rho$) allows us to **give new names** to a relation and its **attributes**. Formally:

$$
\rho_{\,T(a_1,a_2,\ldots,a_n)}(R)
$$

means:
1. Rename the **relation** $R$ to **$T$**,
2. Rename the **attributes** of $R$ to $a_1, a_2, \ldots, a_n$ (in order).

This is useful for:
- Avoiding **name conflicts**, e.g., when two relations use attributes that have the same name but different meanings.
- Improving **readability** of complex expressions (e.g., giving short relation names).

---

### Example. 

If we have:

$$
R(\text{sid, name, major})
$$

and we write

$$
\rho_{\,S(stud\_id, stud\_name, field)}(R),
$$

then:
- $R$ is temporarily called $S$,
- The attributes `(sid, name, major)` become `(stud_id, stud_name, field)`.

---

### SQL Equivalent: Aliases

In SQL, there is no direct “rename operator” command (like $\rho$). Instead, **aliasing** serves a similar role:

1. **Relation alias**: `FROM Students AS S`  
   - Temporary name `S` refers to `Students`.
2. **Attribute alias**: `SELECT R.sid AS stud_id, R.name AS stud_name, R.major AS field ...`

When we combine them, the result is functionally like a **rename** in relational algebra.

---

In [20]:
import sqlite3
from tabulate import tabulate

# 1) Connect to an in-memory SQLite database
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# 2) Create a Students table
cursor.execute("""
    CREATE TABLE Students(
        sid   INTEGER,
        name  TEXT,
        major TEXT
    )
""")

# 3) Insert sample rows
students_data = [
    (1, "Alice", "CS"),
    (2, "Bob",   "Math"),
    (3, "Carol", "CS")
]
cursor.executemany("INSERT INTO Students VALUES (?,?,?)", students_data)

# 4) Display the original table
cursor.execute("SELECT * FROM Students")
original_rows = cursor.fetchall()
print("=== Original Students Table ===")
print(tabulate(original_rows, headers=["sid","name","major"], tablefmt="grid"))

##############################################################################
# 5) Renaming the Table (Alias) & Performing an Operation
##############################################################################
#    We'll alias Students as S, then apply a selection to find only the "CS" majors.

cursor.execute("""
    SELECT 
        S.sid    AS stud_id, 
        S.name   AS stud_name
    FROM Students AS S
    WHERE S.major = 'CS'
""")

filtered_rows = cursor.fetchall()

print("\n=== Selecting from alias 'S' WHERE major = 'CS' ===")
print(tabulate(filtered_rows, headers=["stud_id","stud_name"], tablefmt="grid"))

# 6) Close the connection
conn.close()

=== Original Students Table ===
+-------+--------+---------+
|   sid | name   | major   |
+=======+========+=========+
|     1 | Alice  | CS      |
+-------+--------+---------+
|     2 | Bob    | Math    |
+-------+--------+---------+
|     3 | Carol  | CS      |
+-------+--------+---------+

=== Selecting from alias 'S' WHERE major = 'CS' ===
+-----------+-------------+
|   stud_id | stud_name   |
+===========+=============+
|         1 | Alice       |
+-----------+-------------+
|         3 | Carol       |
+-----------+-------------+
